In [ ]:
# !pip install torch-scatter torch-sparse
!pip install torch-geometric
!pip install torchdata
!pip install igraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.7 MB/s eta 0:00:00


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import pandas as pd

In [ ]:
import gdown # https://drive.google.com/file/d/1sMfFDVqWY39qR8vjDld4zoxDtdSuuoZ8/view?usp=sharing
file_url = "https://drive.google.com/uc?id=1sMfFDVqWY39qR8vjDld4zoxDtdSuuoZ8"
output_path = "books_filtered_df_interaction2.csv"  # Desired output file name
gdown.download(file_url, output_path, quiet=False)
print("File downloaded successfully!")


In [ ]:
book_df = pd.read_csv("books_filtered_df_interaction2.csv")

In [ ]:
book_df.head()

In [ ]:
import pandas as pd

# Example dataframe
df = book_df


In [ ]:
import numpy as np

# Get unique users and products
users = df['reviewerID'].unique()
products = df['asin'].unique()

# Create mappings
user_to_idx = {user: i for i, user in enumerate(users)}
product_to_idx = {product: i + len(users) for i, product in enumerate(products)}

# Total number of nodes
num_nodes = len(users) + len(products)


In [ ]:
import torch

# Create edge list
user_indices = torch.tensor([user_to_idx[user] for user in df['reviewerID']])
product_indices = torch.tensor([product_to_idx[product] for product in df['asin']])

# Combine user and product indices into a single edge index tensor
edge_index = torch.stack([user_indices, product_indices], dim=0)

print("Edge Index:")
print(edge_index)


In [ ]:
# Embedding dimension
embedding_dim = 128

# Initialize random features for all nodes
node_features = torch.randn(num_nodes, embedding_dim)

print("Node Features Shape:", node_features.shape)


In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch_geometric.nn import GCNConv

# # Check if GPU is available and set device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Define the GNN model
# class GNNModel(nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels):
#         super(GNNModel, self).__init__()
#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, out_channels)

#     def forward(self, x, edge_index):
#         # First GCN layer
#         x = self.conv1(x, edge_index)
#         x = F.relu(x)

#         # Second GCN layer
#         x = self.conv2(x, edge_index)
#         return x

# # Initialize model
# embedding_dim = 128  # Example dimension (change as needed)
# hidden_dim = 64
# output_dim = 32
# model = GNNModel(embedding_dim, hidden_dim, output_dim)

# # Move the model to GPU
# model = model.to(device)

# # Initialize optimizer
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# # Example training loop
# num_epochs = 50
# for epoch in range(num_epochs):
#     model.train()
#     optimizer.zero_grad()

#     # Move node features and edge_index to the GPU
#     node_features = node_features.to(device)  # Move node features to GPU
#     edge_index = edge_index.to(device)  # Move edge index to GPU

#     # Forward pass
#     node_embeddings = model(node_features, edge_index)

#     # Define a dummy loss (replace with your clustering loss)
#     loss = F.mse_loss(node_embeddings, node_embeddings)  # Example only
#     loss.backward()
#     optimizer.step()

#     if epoch % 10 == 0:
#         print(f"Epoch {epoch}, Loss: {loss.item()}")


In [ ]:


class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        # First GCN layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        # Second GCN layer
        x = self.conv2(x, edge_index)
        return x

# Initialize model
hidden_dim = 64
output_dim = 32
model = GNNModel(embedding_dim, hidden_dim, output_dim)

print(model)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Check if GPU is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the GNN model
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        # First GCN layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        # Second GCN layer
        x = self.conv2(x, edge_index)
        return x

# Initialize model
embedding_dim = 128  # Example dimension (change as needed)
hidden_dim = 64
output_dim = 32
model = GNNModel(embedding_dim, hidden_dim, output_dim)

# Move the model to GPU
model = model.to(device)

# # Initialize optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


import torch
import torch.nn.functional as F

def contrastive_loss(embeddings, labels, margin=1.0):
    """
    Contrastive loss function.
    embeddings: Tensor of node embeddings.
    labels: Tensor of true labels (0 for same class, 1 for different class).
    margin: Margin for positive pairs.
    """
    loss = 0
    for i in range(len(labels)):
        for j in range(i + 1, len(labels)):
            dist = torch.norm(embeddings[i] - embeddings[j], p=2)  # Distance between pair (i, j)
            if labels[i] == labels[j]:  # Same class, minimize distance
                loss += torch.max(torch.tensor(0.0, device=dist.device), margin - dist)
            else:  # Different class, maximize distance
                loss += dist

    return loss

# Example training loop with contrastive loss
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Move node features and edge_index to GPU
    node_features = node_features.to(device)  # Move node features to GPU
    edge_index = edge_index.to(device)  # Move edge index to GPU

    # Forward pass
    node_embeddings = model(node_features, edge_index)

    # Example labels for user clustering (this should be provided based on your data)
    # Assume we have a label for each node that corresponds to its cluster
    labels = torch.randint(0, 2, (node_embeddings.size(0),))  # Dummy labels: 0 or 1

    # Calculate contrastive loss
    loss = contrastive_loss(node_embeddings, labels)

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

In [ ]:
from sklearn.cluster import KMeans

# Extract user embeddings
user_embeddings = node_embeddings[:len(users)].detach().numpy()

# Dynamically set n_clusters based on the number of users
n_clusters = min(5, len(users))  # Use at most the number of users
kmeans = KMeans(n_clusters=n_clusters)
user_clusters = kmeans.fit_predict(user_embeddings)

# Map users to their clusters
user_cluster_mapping = {user: user_clusters[i] for i, user in enumerate(users)}
print("User Cluster Mapping:", user_cluster_mapping)
